# This notebook was used for genotyping the draft coverage-based CNV calls

In [1]:
import os
import pandas as pd
import numpy as np

from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.notebook import tqdm

# Initialising a bunch of useful dataframes

# Path to coverage-based pipeline's output directory
PATH_TO_EXECUTION = "../assets_pf8/03_execution"

# Loading in Pf8's public metadata and getting QC-passed samples
sample_meta_data = pd.read_csv("../assets_pf8/Pf_8_samples_20241212.txt", sep = "\t", usecols = ["Sample", "QC pass"])
qc_pass_samples  = sample_meta_data.loc[sample_meta_data["QC pass"] == True, "Sample"].tolist()

samples = [sample for sample in os.listdir(os.path.join(PATH_TO_EXECUTION, "postprocessing")) if sample in qc_pass_samples]

# Loading in the regions for which we want to make CNV calls (i.e., the 6 genes)
call_regions_dict = pd.read_csv("../assets_pf8/04_call_regions.tsv", sep = "\t").set_index("CALL_ID").to_dict("index")

# Reading which samples did not have a contig ploidy of 1 for which contigs/chromosomes
failed_test_ploidy = pd.read_csv(os.path.join(PATH_TO_EXECUTION, "failed_test_ploidy.tsv"), sep = "\t", header = None)
failed_test_ploidy.columns = ["SAMPLE", "CONTIG", "PLOIDY", "_"]

failed_train_ploidy = pd.read_csv(os.path.join(PATH_TO_EXECUTION, "failed_test_ploidy.tsv"), sep = "\t", header = None)
failed_train_ploidy.columns = ["SAMPLE", "CONTIG", "PLOIDY", "_"]

failed_ploidy_df = pd.merge(failed_test_ploidy, failed_train_ploidy)

call_regions_dict

{'MDR1': {'CONTIG': 'Pf3D7_05_v3',
  'START': 955955,
  'END': 963095,
  'STRAND': '+',
  'CALL_TYPE': 'CNV'},
 'CRT': {'CONTIG': 'Pf3D7_07_v3',
  'START': 402385,
  'END': 406341,
  'STRAND': '+',
  'CALL_TYPE': 'CNV'},
 'HRP2': {'CONTIG': 'Pf3D7_08_v3',
  'START': 1373212,
  'END': 1376988,
  'STRAND': '-',
  'CALL_TYPE': 'DEL'},
 'GCH1': {'CONTIG': 'Pf3D7_12_v3',
  'START': 974226,
  'END': 976097,
  'STRAND': '+',
  'CALL_TYPE': 'CNV'},
 'HRP3': {'CONTIG': 'Pf3D7_13_v3',
  'START': 2840236,
  'END': 2842840,
  'STRAND': '-',
  'CALL_TYPE': 'DEL'},
 'PM2_PM3': {'CONTIG': 'Pf3D7_14_v3',
  'START': 292244,
  'END': 299101,
  'STRAND': '+',
  'CALL_TYPE': 'CNV'}}

In [2]:
# Setting up the parameters for thresholding
MAX_N_SEGMENTS_ALLOWED = 5
MINIMUM_CONTIG_OCCUPANCY_CN1 = 0.8
MINIMUM_CALL_REGION_OCCUPANCY_FOR_CNV_CALL = 0.75
SUFFICIENT_QS_FOR_CNV_CALL = 400
SUFFICIENT_QA_FOR_CNV_CALL = 40
MINIMUM_RESCUE_WINDOW_OCCUPANCY_CN1_FOR_CNV_RESCUE = 0.8
MINIMUM_CALL_REGION_OCCUPANCY_CN1_FOR_CNV_RESCUE = 0.75

MAXIMUM_CONTIG_MEAN_ABSOLUTE_RESIDUALS = 0.3
MINIMUM_QS_FOR_DEL_CALL = 400

MINIMUM_QS_IN_CN1_REGION_FOR_RESCUE = 1000
MINIMUM_RESCUE_WINDOW_OCCUPANCY_CN1_FOR_DEL_RESCUE = 0.8
MINIMUM_CALL_REGION_OCCUPANCY_CN1_FOR_DEL_RESCUE = 0.2

def _annotate_with_coverage(s: pd.Series, region_start: int, region_end: int) -> float:
    """
    Helper function which calculates how much of the gene of interest is covered by the input segment
    """
    overlap_start = max([s["START"], region_start])
    overlap_end   = min([s["END"],   region_end])
    
    overlap_length = max([0, overlap_end - overlap_start])
    gene_length = (region_end - region_start)
    
    proportion_overlap = overlap_length / gene_length
    
    return proportion_overlap

def _call_del(df: pd.DataFrame, dCR: pd.DataFrame) -> int:
    seg_call_region_subset = df.loc[df.OCCUPANCY_OF_CALL_REGION > 0.].copy()
    
    segment_qc_pass = (seg_call_region_subset.QS >= MINIMUM_QS_FOR_DEL_CALL)
    seg_call_region_qc_pass_subset = seg_call_region_subset.loc[segment_qc_pass]
    
    no_suitable_qc_pass_segments = (len(seg_call_region_qc_pass_subset) == 0)
    acceptable_seg_residual_mean = (dCR.DCR_SEGMENT_RESIDUAL.mean() < MAXIMUM_CONTIG_MEAN_ABSOLUTE_RESIDUALS)

    if no_suitable_qc_pass_segments or not acceptable_seg_residual_mean:
        rescue_window = df.loc[
            (df.CN == 1) & (df.QS >= MINIMUM_QS_IN_CN1_REGION_FOR_RESCUE)]
        
        if rescue_window.SEGMENT_CONTIG_OCCUPANCY.sum() >= MINIMUM_RESCUE_WINDOW_OCCUPANCY_CN1_FOR_DEL_RESCUE:
            rescued_call_df = df.loc[
                (df.OCCUPANCY_OF_CALL_REGION > MINIMUM_CALL_REGION_OCCUPANCY_CN1_FOR_DEL_RESCUE) & (df.CN != 1)
            ].groupby("CN").OCCUPANCY_OF_CALL_REGION.sum().reset_index()
            
            if len(rescued_call_df) > 0:
                return (1 if (rescued_call_df.CN == 0).any() else 0)
            
            else:
                return -1
        
        else:
            return -1
        
    else:
        return (1 if (seg_call_region_qc_pass_subset.CN == 0).any() else 0)

def _call_cnv(df: pd.DataFrame, dCR: pd.DataFrame) -> int:
    seg_call_region_subset = df.loc[df.OCCUPANCY_OF_CALL_REGION >= MINIMUM_CALL_REGION_OCCUPANCY_FOR_CNV_CALL].copy()
    
    if (seg_call_region_subset[["QS", "QA"]].values == 0).any():
        return -1
    
    segment_qc_pass = (seg_call_region_subset.QS >= SUFFICIENT_QS_FOR_CNV_CALL) | (seg_call_region_subset.QA >= SUFFICIENT_QA_FOR_CNV_CALL)
    seg_call_region_qc_pass_subset = seg_call_region_subset.loc[segment_qc_pass]

    no_suitable_qc_pass_segments = (len(seg_call_region_qc_pass_subset) == 0)
    acceptable_seg_residual_mean = (dCR.DCR_SEGMENT_RESIDUAL.mean() < MAXIMUM_CONTIG_MEAN_ABSOLUTE_RESIDUALS)

    if no_suitable_qc_pass_segments or not acceptable_seg_residual_mean:
        rescue_window = df.loc[
            (df.CN == 1) & (df.QS >= MINIMUM_QS_IN_CN1_REGION_FOR_RESCUE)]
        
        if rescue_window.SEGMENT_CONTIG_OCCUPANCY.sum() >= MINIMUM_RESCUE_WINDOW_OCCUPANCY_CN1_FOR_CNV_RESCUE:
            rescued_call_df = df.loc[
                (df.OCCUPANCY_OF_CALL_REGION > MINIMUM_CALL_REGION_OCCUPANCY_CN1_FOR_CNV_RESCUE) & (df.CN != 1)
            ].groupby("CN").OCCUPANCY_OF_CALL_REGION.sum().reset_index()
            
            if len(rescued_call_df) > 0:
                return (1 if (rescued_call_df.CN > 1).any() else 0)
            
            else:
                return -1
        
        else:
            return -1
    else:
        return (1 if (seg_call_region_qc_pass_subset.CN > 1).any() else 0)

def genotype_sample(SAMPLE: str) -> dict:
    sample_results = {CALL_ID: "" for CALL_ID in call_regions_dict.keys()}
    sample_results["SAMPLE"]  = SAMPLE
    
    dCR = pd.read_csv(os.path.join(PATH_TO_EXECUTION, "postprocessing", SAMPLE, f"{SAMPLE}.dCR.tsv"),
                      sep = "\t", skiprows = 18)
    seg = pd.read_csv(os.path.join(PATH_TO_EXECUTION, "postprocessing", SAMPLE, f"{SAMPLE}.segments.vcf.gz"),
                      sep = "\t", skiprows = 35)
    seg["START"] = seg.POS.astype(int)
    seg["END"]   = seg.INFO.apply(lambda s: s.partition("END=")[2]).astype(int)
    
    split_data = seg[SAMPLE].str.split(":").apply(lambda x: tuple(map(int, x)))
    seg[["GT", "CN", "NP", "QA", "QS", "QSE", "QSS"]] = pd.DataFrame(
        [tuple(x) for x in zip(*split_data)]).T
    
    seg = seg[["#CHROM", "START", "END", "CN", "QA", "QS", "QSE", "QSS"]]

    for CALL_ID, call_meta in call_regions_dict.items():
        CALL_REGION_CHROM     = call_meta["CONTIG"]
        CALL_REGION_START     = call_meta["START"]
        CALL_REGION_END       = call_meta["END"]
        CALL_REGION_CALL_TYPE = call_meta["CALL_TYPE"]

        seg_inspection_subset = seg.loc[seg["#CHROM"] == CALL_REGION_CHROM].copy()

        seg_inspection_subset["SEGMENT_LENGTH"] = seg_inspection_subset.END - seg_inspection_subset.START
        seg_inspection_subset["SEGMENT_CONTIG_OCCUPANCY"] = seg_inspection_subset.SEGMENT_LENGTH / seg_inspection_subset.SEGMENT_LENGTH.sum()
        seg_inspection_subset["OCCUPANCY_OF_CALL_REGION"] = seg_inspection_subset.apply(lambda s: 
            _annotate_with_coverage(s, CALL_REGION_START, CALL_REGION_END),
        axis = 1)

        if CALL_REGION_CALL_TYPE == "DEL":
            seg_inspection_subset = seg_inspection_subset.loc[seg_inspection_subset.START <= CALL_REGION_END].reset_index(drop = True)
        
        if len(seg_inspection_subset) > MAX_N_SEGMENTS_ALLOWED:
            sample_results[CALL_ID] = -1
            continue

        contig_ploidy_fail = failed_ploidy_df.loc[
            (failed_ploidy_df.SAMPLE == SAMPLE) & (failed_ploidy_df.CONTIG == CALL_REGION_CHROM)
        ]
        
        if len(contig_ploidy_fail) > 0:
            sample_results[CALL_ID] = -1
            continue
        
        contig_occupancy_grouped_by_cn = seg_inspection_subset.groupby("CN").SEGMENT_CONTIG_OCCUPANCY.sum().reset_index()
        contig_occupancy_cn1 = contig_occupancy_grouped_by_cn.loc[contig_occupancy_grouped_by_cn.CN == 1, "SEGMENT_CONTIG_OCCUPANCY"].values
        
        if len(contig_occupancy_cn1) == 0:
            sample_results[CALL_ID] = -1
            continue
        elif len(contig_occupancy_cn1) > 0:
            if contig_occupancy_cn1[0] < MINIMUM_CONTIG_OCCUPANCY_CN1:
                sample_results[CALL_ID] = -1
                continue
            else:
                pass # Allow calls to proceed if majority of contig is CN=1
        
        for i, row in seg_inspection_subset.iterrows():
            dCR.loc[
                (dCR.CONTIG == row["#CHROM"]) &
                (dCR.START >= row.START) &
                (dCR.END <= row.END),
                "SEGMENT_CN"
            ] = row.CN
        dCR["DCR_SEGMENT_RESIDUAL"] = np.abs(dCR.LINEAR_COPY_RATIO - dCR.SEGMENT_CN)
        dCR = dCR.dropna().copy()

        if CALL_REGION_CALL_TYPE == "DEL":
            sample_results[CALL_ID] = _call_del(seg_inspection_subset, dCR)
            continue
            
        elif CALL_REGION_CALL_TYPE == "CNV":
            sample_results[CALL_ID] = _call_cnv(seg_inspection_subset, dCR)
            continue
        
    return sample_results

In [ ]:
# Highlighting CNV genotyping of a single sample - also helps with bugfixing to be able to just run on a single sample
genotype_sample("SPT00899")

{'MDR1': 0,
 'CRT': 0,
 'HRP2': 0,
 'GCH1': 0,
 'HRP3': 1,
 'PM2_PM3': 0,
 'SAMPLE': 'SPT00899'}

In [ ]:
# Multiprocessing for CNV genotyping all samples
with ProcessPoolExecutor(max_workers = 64) as executor:
    list_of_results = []
    
    futures = {executor.submit(genotype_sample, sample): sample for sample in samples}

    with tqdm(total = len(samples)) as pbar:
        for future in as_completed(futures):            
            list_of_results.append(future.result())
            pbar.update(1)

# Export draft calls as `draft_coverage_calls.tsv`
df = pd.DataFrame(list_of_results).sort_values("SAMPLE").reset_index(drop = True)
df[["SAMPLE"] + [CALL_ID for CALL_ID in call_regions_dict.keys()]].to_csv("app_files/draft_coverage_calls.tsv", sep = "\t", index = False)

  0%|          | 0/24409 [00:00<?, ?it/s]

In [3]:
# Loading in `draft_coverage_calls.tsv`
df = pd.read_csv("app_files/draft_coverage_calls.tsv", sep = "\t")

In [ ]:
# Combining with Pf7's drug resistance file to check for concordance
pd.set_option("display.max.colwidth", None)

pf7_resistance = pd.read_csv(
    "app_files/Pf7_inferred_resistance_status_classification.tsv", sep = "\t"
).set_index("Sample")[
    ["Mefloquine", "Piperaquine", "HRP2", "HRP3"]
].map(lambda val: {
    "Sensitive"   :  "0",
    "Resistant"   :  "1",
    "Undetermined": "-1",
    "nodel"       :  "0",
    "del"         :  "1",
    "uncallable"  : "-1"
    }[val]
).rename(columns = {
    "Mefloquine" : "Pf7Final:MDR1",
    "Piperaquine": "Pf7Final:PM2_PM3",
    "HRP2"       : "Pf7Final:HRP2",
    "HRP3"       : "Pf7Final:HRP3"
})

meta = pd.read_csv("../assets_pf8/Pf_8_samples_20241212.txt", sep = "\t", usecols = ["Sample", "QC pass", "Sample was in Pf7"]).set_index("Sample")

merged = (
    pf7_resistance.join(meta, how = "outer")
    .join(df.set_index("SAMPLE"), how = "outer")
    .reset_index()
    .rename(columns = {"index": "Sample"})
    .fillna("MISSING")
    .map(lambda x: (int(x) if type(x) == float else x))
)

merged = merged.loc[
    (merged["QC pass"] == True)
    # & (merged["Sample was in Pf7"] == True)
    # & ~(merged["Sample was in Pf7"] == True)
]

n_total_samples = len(merged)
print(n_total_samples)

n_example_samples = 100

24409


---

# Check concordance for one gene at a time.

#### Python lists here can be copy and pasted directly into `app.py` for rapid inspection

#### "MISSING" on the first column (e.g., Pf7Final:MDR1) implies these samples were not in Pf7 and were new in Pf8.

In [10]:
merged.groupby(["Pf7Final:MDR1", "MDR1"]).apply(lambda s: pd.Series({
    "COUNT": len(s.Sample),
    "PERCENTAGE": f"{len(s.Sample) / n_total_samples:.2%}",
    "EXAMPLES": '["' + '", "'.join(s.Sample.sample(min([n_example_samples, len(s.Sample)]))) + '"]'
}), include_groups = False)

COUNT PERCENTAGE  \
Pf7Final:MDR1 MDR1                     
-1            -1     1380      5.65%   
               0       30      0.12%   
               1        7      0.03%   
0             -1     3933     16.11%   
               0     9934     40.70%   
               1       15      0.06%   
1             -1       41      0.17%   
               0       17      0.07%   
               1      642      2.63%   
MISSING       -1     5755     23.58%   
               0     2636     10.80%   
               1       19      0.08%   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   EXAMPLES  
Pf7Final:MDR1 MDR1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
-1            -1    ["SPT36128", "SPT00202", "SPT41911", "SPT24493", "RCN08675", "SPT24651", "SPT24513", "SPT16953", "SPT19356", "SPT25034", "RCN03360", "RCN09292", "SPT19947", "SPT40589", "PM0034-C", "SPT40816", "SPT35133", "SPT17972", "SPT19872", "SPT35519", "SPT24628", "RCN13907", "RCN10308", "SPT15621", "SPT19521", "SPT24826", "SPT19681", "RCN02656", "SPT40623", "PH1678-CW2", "SPT24630", "SPT40847", "SPT34453", "SPT36356", "SPT21295", "SPT36104", "PA0330-CW", "SPT19396", "SPT36713", "SPT19764", "SPT24790", "SPT36671", "SPT35587", "RCN09561", "RCN02544", "SPT36612", "SPT19341", "SPT41999", "QG0364-C", "SPT12505", "SPT12492", "QG0439-C", "RCN02974", "SPT17924", "PA1062-CW", "RCN02767", "SPT22438", "SPT40676", "QG0353-C", "SPT36877", "PF0046-C", "RCN02993", "SPT38372", "SPT19383", "SPT15529", "RCN11270", "SPT19024", "RCN09045", "RCN03160", "SPT40943", "SPT12524", "SPT24526", "SPT24738", "SPT36127", "SPT41939", "SPT36057", "SPT35590", "SPT35537", "SPT34234", "SPT38319", "RCN03099", "S

In [11]:
merged.groupby(["Pf7Final:PM2_PM3", "PM2_PM3"]).apply(lambda s: pd.Series({
    "COUNT": len(s.Sample),
    "PERCENTAGE": f"{len(s.Sample) / n_total_samples:.2%}",
    "EXAMPLES": '["' + '", "'.join(s.Sample.sample(min([n_example_samples, len(s.Sample)]))) + '"]'
}), include_groups = False)

COUNT PERCENTAGE  \
Pf7Final:PM2_PM3 PM2_PM3                     
-1               -1        1450      5.94%   
                  0          10      0.04%   
                  1           2      0.01%   
0                -1        3075     12.60%   
                  0       10181     41.71%   
                  1          14      0.06%   
1                -1         369      1.51%   
                  0         417      1.71%   
                  1         481      1.97%   
MISSING          -1        5518     22.61%   
                  0        2868     11.75%   
                  1          24      0.10%   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        EXAMPLES  
Pf7Final:PM2_PM3 PM2_PM3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
-1               -1           ["RCN14151", "SPT15617", "SPT38343", "SPT34411", "RCN11208", "SPT19351", "SPT18492", "SPT36877", "SPT42023", "SPT26533", "SPT34801", "SPT36375", "SPT40750", "SPT36344", "SPT24953", "SPT25002", "SPT26891", "SPT42003", "SPT19931", "RCN02999", "SPT40943", "SPT35673", "RCN11215", "SPT19494", "RCN00768", "SPT24760", "RCN11111", "RCN02846", "SPT15535", "SPT40733", "RCN12829", "SPT18991", "RCN03174", "SPT24581", "RCN02282", "SPT20000", "SPT12097", "SPT35556", "RCN02288", "SPT41903", "SPT40816", "SPT36264", "SPT22566", "SPT12098", "PA1456-Cw", "SPT16875", "SPT36317", "RCN01045", "SPT24613", "RCN09741", "SPT12863", "SPT15184", "SPT00192", "SPT00035", "SPT36072", "SPT24587", "SPT17929", "SPT21807", "SPT41089", "RCN14091", "SPT20565", "SPT35516", "RCN02961", "SPT17972", "PH0713-C", "SPT15599", "RCN10014", "SPT17950", "RCN11892", "SPT34255", "SPT41094", "SPT17812", "SPT42020", "SP

In [12]:
merged.groupby(["Pf7Final:HRP2", "HRP2"]).apply(lambda s: pd.Series({
    "COUNT": len(s.Sample),
    "PERCENTAGE": f"{len(s.Sample) / n_total_samples:.2%}",
    "EXAMPLES": '["' + '", "'.join(s.Sample.sample(min([n_example_samples, len(s.Sample)]))) + '"]'
}), include_groups = False)

COUNT PERCENTAGE  \
Pf7Final:HRP2 HRP2                     
-1            -1     4346     17.80%   
               0      766      3.14%   
               1        8      0.03%   
0             -1      891      3.65%   
               0     9951     40.77%   
               1       22      0.09%   
1             -1        3      0.01%   
               0        1      0.00%   
               1       11      0.05%   
MISSING       -1     5545     22.72%   
               0     2795     11.45%   
               1       70      0.29%   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  EXAMPLES  
Pf7Final:HRP2 HRP2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
-1            -1    ["RCN13950", "SPT25116", "SPT18549", "SPT34281", "SPT12513", "SPT34273", "SPT35780", "RCN12113", "RCN02894", "RCN09678", "SPT34904", "RCN14055", "SPT36233", "PA0313-CW", "PA0916-CW", "SPT40861", "RCN09960", "SPT15586", "SPT36190", "SPT00203", "SPT24826", "SPT20030", "SPT42042", "SPT40988", "SPT36285", "RCN10727", "SPT13073", "RCN02281", "RCN00879", "SPT34469", "SPT35245", "RCN08955", "RCN03433", "SPT26864", "RCN03178", "RCN14030", "SPT20499", "SPT19824", "SPT35116", "SPT19385", "SPT34451", "RCN14017", "PM0752-CW", "PF1066-Cx", "SPT17050", "SPT34261", "PC0077-C", "RCN00871", "SPT35241", "RCN11232", "RCN14137", "SPT34866", "SPT24724", "RCN03165", "RCN02272", "SPT17918", "SPT35437", "SPT35571", "SPT12074", "SPT24743", "RCN12809", "QC0420-CW", "RCN13375", "SPT12367", "RCN02843", "SPT41280", "SPT35036", "RCN12063", "SPT35580", "SPT41036", "SPT12990", "RCN08722", "SPT19875", "SPT19382", "RCN09010", "SPT12292", "SPT36682", "RCN09940", "SPT24141", "SPT17057", "SPT26304", "QG

In [13]:
merged.groupby(["Pf7Final:HRP3", "HRP3"]).apply(lambda s: pd.Series({
    "COUNT": len(s.Sample),
    "PERCENTAGE": f"{len(s.Sample) / n_total_samples:.2%}",
    "EXAMPLES": '["' + '", "'.join(s.Sample.sample(min([n_example_samples, len(s.Sample)]))) + '"]'
}), include_groups = False)

COUNT PERCENTAGE  \
Pf7Final:HRP3 HRP3                     
-1            -1     4654     19.07%   
               0      879      3.60%   
               1       18      0.07%   
0             -1      674      2.76%   
               0     9563     39.18%   
               1       13      0.05%   
1             -1       11      0.05%   
               0        1      0.00%   
               1      186      0.76%   
MISSING       -1     5717     23.42%   
               0     2678     10.97%   
               1       15      0.06%   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   EXAMPLES  
Pf7Final:HRP3 HRP3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
-1            -1      ["SPT12299", "RCN08753", "SPT40959", "SPT26446", "SPT35092", "RCN13918", "SPT19238", "SPT36185", "SPT34471", "SPT17956", "SPT19600", "QC0402-CW", "SPT35592", "SPT15635", "SPT24529", "PF1106-Cx", "SPT34321", "SPT40541", "SPT35540", "RCN08889", "RCN11315", "SPT40956", "PF0720-C", "SPT36106", "PW0090-C", "SPT38380", "SPT12556", "SPT18609", "SPT34631", "SPT17830", "RCN02366", "SPT36703", "PH0747-Cx", "SPT36613", "SPT24740", "RCN14066", "SPT42003", "SPT35005", "RCN09990", "SPT36600", "SPT20013", "RCN12550", "SPT19851", "SPT36886", "SPT23019", "SPT18428", "SPT20563", "RCN08961", "SPT38353", "RCN03094", "SPT36875", "SPT40804", "SPT19251", "SPT19766", "RCN14122", "SPT35573", "SPT17046", "SPT00192", "SPT22407", "PM0774-CW", "RCN14042", "SPT12076", "SPT22480", "SPT19415", "SPT17078", "SPT18932", "RCN09283", "RCN11621", "RCN09812", "RCN13480", "SPT18431", "SPT22608", "RCN13950", "SPT34629", "SPT19419", "RCN03337", "SPT25103", "SPT17993", "SPT15640", "SPT36126", "RCN02030", 

In [14]:
merged.groupby(["GCH1"]).apply(lambda s: pd.Series({
    "COUNT": len(s.Sample),
    "PERCENTAGE": f"{len(s.Sample) / n_total_samples:.2%}",
    "EXAMPLES": '["' + '", "'.join(s.Sample.sample(min([n_example_samples, len(s.Sample)]))) + '"]'
}), include_groups = False)

,COUNT,PERCENTAGE,EXAMPLES
GCH1,,,
-1,11561,47.36%,"[""RCN13959"", ""RCN00223"", ""SPT15260"", ""SPT21321"", ""PM0190-C"", ""RCN13377"", ""SPT12519"", ""SPT34359"", ""RCN12613"", ""RCN23115"", ""SPT40538"", ""SPT15545"", ""RCN11434"", ""SPT67042"", ""SPT83123"", ""PC0122-C"", ""SPT46503"", ""SPT23059"", ""RCN03290"", ""SPT34771"", ""SPT88322"", ""RCN22931"", ""SPT50431"", ""RCN09667"", ""SPT91676"", ""SPT72018"", ""SPT24885"", ""RCN02142"", ""PA0342-CW"", ""SPT16959"", ""RCN22990"", ""RCN12820"", ""RCN03528"", ""SPT35538"", ""RCN02242"", ""SPT54654"", ""RCN02702"", ""SPT20567"", ""SPT46285"", ""SPT34471"", ""RCN22790"", ""RCN18536"", ""SPT38752"", ""SPT19503"", ""PA0295-CW"", ""RCN11425"", ""SPT19521"", ""RCN00972"", ""SPT83807"", ""SPT16871"", ""RCN01064"", ""SPT70686"", ""SPT34691"", ""RCN11438"", ""RCN09654"", ""RCN25059"", ""RCN09914"", ""SPT83142"", ""SPT34492"", ""SPT66979"", ""SPT26859"", ""SPT65173"", ""SPT18177"", ""RCN02538"", ""SPT26935"", ""SPT84123"", ""SPT41017"", ""RCN13931"", ""SPT52971"", ""RCN11221"", ""SPT83790"", ""SPT83628"", ""SPT67118"", ""SPT34235"", ""SPT54435"", ""RCN09016"", ""SPT00203"", ""SPT18434"", ""RCN22737"", ""SPT44885"", ""QP0166-C"", ""SPT41055"", ""RCN08692"", ""SPT65141"", ""SPT44018"", ""SPT19746"", ""QC0536-CW"", ""SPT40776"", ""SPT65241"", ""SPT88095"", ""SPT43785"", ""SPT34892"", ""SPT45024"", ""SPT53093"", ""RCN03311"", ""PW0045-C"", ""SPT53731"", ""RCN26058"", ""SPT43733"", ""RCN08831""]"
0,11095,45.45%,"[""PK0068-C"", ""RCN11927"", ""RCN13069"", ""RCN01074"", ""RCN00771"", ""PJ0270-C"", ""PD0577-C"", ""RCN10364"", ""RCN10377"", ""SPT50169"", ""PF0156-C"", ""PH1713-C"", ""RCN15167"", ""SPT34445"", ""SPT50823"", ""PM0249-C"", ""SPT44045"", ""PM0541-CW"", ""RCN18675"", ""RCN08922"", ""PF0311-C"", ""QQ0054-C"", ""RCN15243"", ""SPT14951"", ""SPT50325"", ""RCN10203"", ""SPT53903"", ""PM0132-C"", ""SPT53821"", ""SPT16755"", ""RCN00290"", ""PC0013-C"", ""PA0544-C"", ""QG0094-C"", ""SPT50241"", ""RCN08330"", ""SPT53849"", ""RCN13499"", ""PE0148-C"", ""RCN18745"", ""SPT67180"", ""PH0933-C"", ""SPT43032"", ""RCN02589"", ""FP0039-C"", ""PC0206-C"", ""SPT16995"", ""RCN00215"", ""PM0758-CW"", ""RCN24933"", ""PW0054-C"", ""QG0069-C"", ""SPT16605"", ""SPT51160"", ""PT0294-C"", ""PD0540-C"", ""PH0475-C"", ""PA0208-C"", ""RCN09007"", ""PM0320-C"", ""PH0962-C"", ""PD0774-C"", ""PF0117-C"", ""SPT54666"", ""RCN12797"", ""SPT50273"", ""PV0176-C"", ""SPT16788"", ""SPT42905"", ""PF1030-C"", ""SPT67571"", ""PA0182-C"", ""PH1339-C"", ""QQ0088-C"", ""RCN09116"", ""PH0528-C"", ""PM0809-C"", ""SPT43284"", ""RCN11196"", ""RCN08019"", ""RCN10199"", ""QG0192-C"", ""SPT46151"", ""QC0326-C"", ""RCN03125"", ""SPT51218"", ""PH1377-C"", ""PM0332-C"", ""PE0127-C"", ""QP0156-C"", ""SPT53784"", ""RCN15005"", ""SPT24153"", ""PT0006-CW"", ""SPT67032"", ""PM0413-C"", ""RCN12461"", ""PF1097-C"", ""PF0557-C"", ""QP0093-C""]"
1,1753,7.18%,"[""QC0138-C"", ""SPT15137"", ""RCN00209"", ""PH0549-C"", ""PD0515-C"", ""PD1093-C"", ""RCN09360"", ""PF0925-C"", ""PD1028-C"", ""PF0476-C"", ""PD0626-C"", ""PR0264-C"", ""QC0115-C"", ""RCN00253"", ""QC0324-C"", ""PD1087-C"", ""PD0915-C"", ""PF0955-C"", ""PH0327-C"", ""PD1135-C"", ""RCN09826"", ""RCN09889"", ""QP0110-C"", ""RCN03019"", ""PD1017-C"", ""RCN09317"", ""PH0346-C"", ""SPT15127"", ""QC0330-C"", ""PD0717-C"", ""PD0925-C"", ""SPT15052"", ""PH0014-C"", ""QE0464-C"", ""QP0026-C"", ""PD1185-C"", ""RCN00968"", ""RCN09863"", ""QP0041-C"", ""PD0862-C"", ""PD1086-C"", ""QC0129-C"", ""RCN02049"", ""RCN02315"", ""RCN08388"", ""PD0094-C"", ""QP0254-C"", ""SPT43918"", ""SPT36913"", ""PH0900-C"", ""PD0056-C"", ""PD0771-C"", ""RCN08207"", ""PR0125-C"", ""PD1099-C"", ""RCN08191"", ""SPT41047"", ""PH0382-C"", ""RCN08371"", ""QP0230-C"", ""PD0835-C"", ""RCN00291"", ""PR0016-C"", ""RCN00280"", ""SPT41193"", ""PD1007-C"", ""PH0569-C"", ""PF1086-C"", ""RCN02369"", ""PD1143-C"", ""RCN09929"", ""RCN09868"", ""RCN02558"", ""PD1019-C"", ""SPT15077"", ""RCN09316"", ""SPT67645"", ""PF0262-C"", ""PJ0247-C"", "

In [15]:
merged.groupby(["CRT"]).apply(lambda s: pd.Series({
    "COUNT": len(s.Sample),
    "PERCENTAGE": f"{len(s.Sample) / n_total_samples:.2%}",
    "EXAMPLES": '["' + '", "'.join(s.Sample.sample(min([n_example_samples, len(s.Sample)]))) + '"]'
}), include_groups = False)

,COUNT,PERCENTAGE,EXAMPLES
CRT,,,
-1,10671,43.72%,"[""SPT43489"", ""SPT44504"", ""PA1061-CW"", ""RCN02254"", ""RCN16528"", ""SPT44869"", ""RCN25349"", ""SPT84014"", ""RCN09401"", ""SPT40576"", ""RCN11229"", ""SPT18986"", ""RCN11613"", ""SPT72368"", ""QC0572-CW"", ""SPT34526"", ""RCN14682"", ""SPT12304"", ""SPT44655"", ""SPT54523"", ""SPT18194"", ""RCN08626"", ""PA0284-CW"", ""SPT44832"", ""SPT26522"", ""SPT40804"", ""PF1236-CW"", ""RCN09854"", ""SPT19955"", ""RCN01874"", ""RCN25948"", ""RCN02308"", ""SPT24778"", ""RCN23112"", ""RCN12359"", ""RCN07837"", ""SPT54418"", ""SPT26558"", ""RCN12370"", ""SPT18585"", ""SPT54614"", ""SPT52649"", ""SPT51180"", ""SPT38422"", ""SPT44676"", ""SPT72087"", ""SPT91140"", ""SPT88346"", ""SPT64700"", ""SPT55525"", ""SPT40838"", ""SPT67816"", ""SPT72019"", ""QC0557-CW"", ""RCN23084"", ""PV0297-C"", ""SPT67846"", ""SPT22248"", ""SPT38302"", ""SPT18604"", ""RCN15082"", ""SPT19753"", ""SPT38429"", ""SPT18955"", ""SPT88596"", ""SPT26950"", ""SPT18971"", ""RCN13086"", ""SPT19601"", ""SPT51378"", ""SPT91254"", ""RCN02925"", ""SPT51216"", ""RCN03353"", ""SPT15253"", ""SPT54466"", ""SPT53644"", ""SPT71983"", ""SPT91004"", ""SPT90752"", ""SPT26940"", ""SPT44692"", ""SPT70570"", ""SPT34791"", ""SPT24604"", ""RCN22612"", ""RCN02902"", ""SPT36593"", ""SPT24449"", ""SPT52437"", ""SPT67754"", ""SPT36384"", ""SPT44308"", ""SPT20030"", ""RCN25320"", ""SPT70964"", ""SPT54727"", ""SPT67491"", ""PF0670-C"", ""SPT42039""]"
0,13677,56.03%,"[""RCN08017"", ""PF0529-C"", ""PF0202-C"", ""PK0007-C"", ""SPT43192"", ""SPT43357"", ""SPT50352"", ""RCN11670"", ""RCN24810"", ""SPT46563"", ""RCN14667"", ""QP0202-C"", ""PF0583-C"", ""PD1464-C"", ""PD0726-C"", ""PM0206-C"", ""RCN11192"", ""RCN11775"", ""PD1399-C"", ""PW0002-C"", ""PD0772-C"", ""RCN25018"", ""RCN14721"", ""QG0143-C"", ""RCN00225"", ""SPT15133"", ""RCN08042"", ""SPT43330"", ""SPT67618"", ""PC0013-C"", ""SPT66905"", ""SPT35207"", ""SPT40677"", ""PM0204-C"", ""SPT43077"", ""PA0488-C"", ""RCN00261"", ""SPT67661"", ""PH0626-C"", ""SPT46152"", ""SPT43016"", ""PE0393-C"", ""SPT34896"", ""RCN14592"", ""QG0380-C"", ""QT0028-CW5"", ""PF0430-C"", ""RCN09062"", ""PF0010-C"", ""SPT36179"", ""RCN00211"", ""PM0565-C"", ""RCN08289"", ""PE0468-C"", ""QG0192-C"", ""PA0217-C"", ""QP0054-C"", ""PE0180-C"", ""SPT50856"", ""RCN10400"", ""QG0011-C"", ""RCN14937"", ""QT0033-CW"", ""SPT43085"", ""RCN08270"", ""QC0364-C"", ""PF0651-C"", ""SPT41106"", ""PV0257-C"", ""FP0073-C"", ""RCN08261"", ""QQ0049-C"", ""PF0874-C"", ""RCN08326"", ""PM0144-C"", ""SPT50887"", ""SPT15071"", ""RCN11625"", ""SPT15695"", ""SPT50258"", ""PT0061-C"", ""SPT34416"", ""PW0090-C"", ""PF0026-C"", ""PF0835-C"", ""QC0231-C"", ""PM0093-C"", ""PD1361-C"", ""SPT41870"", ""PJ0162-C"", ""PR0145-C"", ""PE0253-C"", ""PH1742-C"", ""RCN02744"", ""RCN24926"", ""PD0992-C"", ""RCN15041"", ""SPT22449"", ""SPT67545"", ""SPT83112""]"
1,61,0.25%,"[""PK0063-C"", ""SPT42947"", ""PF0905-C"", ""RCN13280"", ""RCN09132"", ""PF0219-C"", ""SPT22565"", ""PM0496-C"", ""SPT46144"", ""PK0051-C"", ""PF0729-C"", ""PF0195-C"", ""RCN15335"", ""PK0068-C"", ""PF0392-C"", ""PM0455-C"", ""PF0356-C"", ""PF0934-C"", ""PM0345-C"", ""PF0118-C"", ""PM0243-C"", ""PF0190-C"", ""SPT41222"", ""SPT26638"", ""PF0796-C"", ""PF0987-C"", ""PK0069-C"", ""SPT00802"", ""PK0019-C"", ""PF0149-C"", ""PF1103-C"", ""PF0054-C"", ""PK0023-C"", ""PM0336-C"", ""PM0124-C"", ""PM0580-C"", ""PF0960-C"", ""PM0548-CW"", ""PF0449-C"", ""PK0046-C"", ""RCN08145"", ""PF0231-C"", ""RCN11806"", ""RCN09082"", ""PM0476-C"", ""RCN08130"", ""SPT16920"", ""QV0096-C"", ""SPT26595"", ""RCN15176"", ""RCN15085"", ""PM0313-C"", ""PF0852-C"", ""PF0250-C"", ""RCN15231"", ""PK0066-C"", ""PF0590-C"", ""PA0193-C"", ""PF0961-C"", ""PK0075-C"", ""QQ0097-C""]"


---

# Discordance mainly in PM2_PM3